$C(S,t)= N(d_1)S - N(d_2)Ke^{-r(T-t)}$

$d_1= \frac{1}{s\sqrt{T-t}} [\ln(\frac{S}{K})+ (r+\frac{S^2}{2})(T-t)]$

$d_2= d_1- S\sqrt{T-t}$

In [19]:
import numpy as np
import pandas as pd
from pandas_datareader import data as wb
from scipy.stats import norm

$d_1= \frac{\ln(\frac{S}{K})+(r+ \frac{{stdev}^2}{2})t}{s\sqrt{t}}$

$d_2= d_1- s\sqrt{t}= \frac{\ln(\frac{S}{K})+(r- \frac{{stdev}^2}{2})t}{s\sqrt{t}}$

where $S$ - stock price, $K$ - strike price, $r$ - risk-free rate, $stdev$ - standard deviation, $T$ - time horizon (years).

$C(S,t)= SN(d_1) - Ke^{-rt}N(d_2)$

In [20]:
def d1(S, K, r, stdev, T):
    return (np.log(S/K) + (r + stdev**2 / 2) * T) / (stdev * np.sqrt(T))

def d2(S, K, r, stdev, T):
    return (np.log(S/K) + (r - stdev**2 / 2) * T) / (stdev * np.sqrt(T))

We need a cumulative normal distribution. Cumulative Distribution Function shows how data accumulates in time. Resulting values are in $[0,1]$. It shows what portion of values lies below the specified value.

In [21]:
print(norm.cdf(0))
print(norm.cdf(0.5))
print(norm.cdf(0.9))

0.5
0.6914624612740131
0.8159398746532405


In [22]:
def BSM(S, K, r, stdev, T):
    return (S * norm.cdf(d1(S, K, r, stdev, T))) - (K * np.exp(-r * T) * norm.cdf(d2(S, K, r, stdev, T)))

In [23]:
ticker = 'PG'
data = pd.DataFrame()
data[ticker] = wb.DataReader(ticker, data_source='yahoo', start='2011-1-1', end='2021-3-8')['Adj Close']

In [24]:
S = data.iloc[-1]
S

PG    125.980003
Name: 2021-03-05 00:00:00, dtype: float64

In [25]:
log_returns = np.log(1 + data.pct_change())
stdev = log_returns.std() * 250 ** 0.5
stdev

PG    0.174309
dtype: float64

In [26]:
r = 0.025  # 2.5% is the yield of 10 year government bond
K = 110.0
T = 1

In [29]:
# Call option price
BSM(S, K, r, stdev, T)

PG    20.647217
Name: 2021-03-05 00:00:00, dtype: float64

Call option price is much lower than a stock price:

In [31]:
print(BSM(S, K, r, stdev, T))
print(S)

PG    20.647217
Name: 2021-03-05 00:00:00, dtype: float64
PG    125.980003
Name: 2021-03-05 00:00:00, dtype: float64
